<a href="https://colab.research.google.com/github/rukmals/crowd-monitoring-system-model-development/blob/main/GT_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import os
#from tensorflow.keras.preprocessing import image
import numpy as np
import scipy
from scipy.io import loadmat
import glob
import h5py
import time
import joblib
from joblib import Parallel, delayed
import sys
import scipy.spatial
import math
import tensorflow as tf
from tensorflow.keras.preprocessing import image

print("Start")

from google.colab import drive
drive.mount('/content/drive')

Start
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
path_list_file = "/content/drive/MyDrive/GCC/all_list.txt"
def get_image_path(file_path):
    file_path_list = file_path.split(" ")
    scene = file_path_list[3][4:]
    image_number = file_path_list[4]
    image_path = "/content/drive/MyDrive/GCC/"+"Part"+" "+scene[7]+scene+"/"+"pngs/"+image_number+".png"
    return image_path

file = open(path_list_file, 'r')

file_list = file.readlines()

image_path_list = []

for line in file_list:
    image_path_list.append(get_image_path(line))

print(len(image_path_list))


def get_image_pathlist(path_list, part):
    image_path_list_part_ = []
    for line_ in path_list:
        if line_.find(part)!=-1:
            image_path_list_part_.append(line_)
    return image_path_list_part_

part_list = get_image_pathlist(image_path_list, "Part 1")
print(len(part_list))

def gaussian_filter_density(gt):
    #print(gt.shape)
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density

    pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
    leafsize = 2048
    # build kdtree
    pts_copy = pts.copy()
    tree = scipy.spatial.KDTree(pts_copy, leafsize=leafsize)
    # query kdtree
    distances, locations = tree.query(pts, k=4)
    print(gt_count)
    #print(distances)
    #print(locations)
    print('generate density...')
    count = 0
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1], pt[0]] = 1.
        #print(gt_count)
        if gt_count > 1 and math.isinf(distances[i][1])==False and math.isinf(distances[i][2])==False and math.isinf(distances[i][3])==False:
            #print(distances)
            sigma = (distances[i][1] + distances[i][2]+distances[i][3]) * 0.1
            count+=1
            
        elif gt_count > 1 and math.isinf(distances[i][1])==False and math.isinf(distances[i][2])==False and math.isinf(distances[i][3])==True :
            #print(distances)
            sigma = (distances[i][1] + distances[i][2]) * 0.1
            count+=1

            #print(sigma) 
        elif gt_count > 1 and math.isinf(distances[i][1])==False and math.isinf(distances[i][2])==True and math.isinf(distances[i][3])==False:
            
            sigma = (distances[i][1] + distances[i][3]) * 0.1
            count+=1

        elif gt_count > 1 and math.isinf(distances[i][1])==False and math.isinf(distances[i][2])==True and math.isinf(distances[i][3])==True:
            
            sigma = (distances[i][1]) * 0.1
            count+=1

        else:
            sigma = np.average(np.array(gt.shape)) / 2. / 2.  # case: 1 point
            count+=1

        #print(sigma)
        density += scipy.ndimage.filters.gaussian_filter(pt2d, round(sigma), mode='constant')
    print('done.')
    #print(count)
    #print(np.sum(density))
    return density

def generate_density_map(img_path):
    #print(img_path)
    #print('mat_path ', mat_path)
    mat_path = img_path.replace("pngs","mats").replace(".png",".mat")
    mat = scipy.io.loadmat(mat_path)
    imgfile = image.load_img(img_path)
    img = image.img_to_array(imgfile)
    k = np.zeros((img.shape[0], img.shape[1]))
    img_info = mat["image_info"]
    #weather_info = mat["weather_info"]
    #time_info = mat["time_info"]
    #roi_info = mat["roi_info"]
    #camera_info = mat["camera_info"]
    gt = img_info[0][0][0]
    for i in range(0, len(gt)):
        #print(gt[i][0])
        #print(gt[i])
        if int(gt[i][0]) < img.shape[0] and int(gt[i][1]) < img.shape[1]:
            #print(gt[i][1],gt[i][0])
            k[int(gt[i][0]), int(gt[i][1])] = 1
    k = gaussian_filter_density(k)
    #print(k)
    output_path = img_path.replace("pngs", "GT").replace('.png', '.h5')
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)
    print(output_dir)
    print("output", output_path)
    sys.stdout.flush()
    with h5py.File(output_path, 'w') as hf:
        hf['density'] = k
        hf.close()
    return img_path
    #return img_info

#mat_path = "D:/Aca sem 6/FYP/DataSets/GCC/Part 6/scene_60_0/mats/1532367537.mat"
#img_path = "/content/drive/MyDrive/scene_09_2/pngs/1531113494.png"
#for img_path in part_0_list[240]:
#generate_density_map(part_0_list[240])

15212
1545


In [21]:
len(part_list)

1545

In [22]:
part_list[0:41]

['/content/drive/MyDrive/GCC/Part 1/scene_10_0/pngs/1531114578.png',
 '/content/drive/MyDrive/GCC/Part 1/scene_10_0/pngs/1531114596.png',
 '/content/drive/MyDrive/GCC/Part 1/scene_10_0/pngs/1531114611.png',
 '/content/drive/MyDrive/GCC/Part 1/scene_10_0/pngs/1531114630.png',
 '/content/drive/MyDrive/GCC/Part 1/scene_10_0/pngs/1531114672.png',
 '/content/drive/MyDrive/GCC/Part 1/scene_10_0/pngs/1531114700.png',
 '/content/drive/MyDrive/GCC/Part 1/scene_10_0/pngs/1531114709.png',
 '/content/drive/MyDrive/GCC/Part 1/scene_10_0/pngs/1531114737.png',
 '/content/drive/MyDrive/GCC/Part 1/scene_10_0/pngs/1531114751.png',
 '/content/drive/MyDrive/GCC/Part 1/scene_10_0/pngs/1531114771.png',
 '/content/drive/MyDrive/GCC/Part 1/scene_10_0/pngs/1531114808.png',
 '/content/drive/MyDrive/GCC/Part 1/scene_10_0/pngs/1531114832.png',
 '/content/drive/MyDrive/GCC/Part 1/scene_10_0/pngs/1531114854.png',
 '/content/drive/MyDrive/GCC/Part 1/scene_10_0/pngs/1531114875.png',
 '/content/drive/MyDrive/GCC/Part 

In [27]:
import threading
import time
start = time.time()
threading_list = part_list[0:40]
for x in range(0,len(threading_list),10):
  print(x)
  t1 = threading.Thread(target = generate_density_map, args = (threading_list[x],))
  t2 = threading.Thread(target = generate_density_map, args = (threading_list[x+1],))
  t3 = threading.Thread(target = generate_density_map, args = (threading_list[x+2],))
  t4 = threading.Thread(target = generate_density_map, args = (threading_list[x+3],))
  t5 = threading.Thread(target = generate_density_map, args = (threading_list[x+4],))
  t6 = threading.Thread(target = generate_density_map, args = (threading_list[x+5],))
  t7 = threading.Thread(target = generate_density_map, args = (threading_list[x+6],))
  t8 = threading.Thread(target = generate_density_map, args = (threading_list[x+7],))
  t9 = threading.Thread(target = generate_density_map, args = (threading_list[x+8],))
  t10 = threading.Thread(target = generate_density_map, args = (threading_list[x+9],))

  t1.start()
  t2.start()
  t3.start()
  t4.start()
  t5.start()
  t6.start()
  t7.start()
  t8.start()
  t9.start()
  t10.start()

  t1.join()
  t2.join()
  t3.join()
  t4.join()
  t5.join()
  t6.join()
  t7.join()
  t8.join()
  t9.join()
  t10.join()
  

  print("One Iteration Over!!")

print("All Iterations Done!!!")
end = time.time()
print(f"Runtime of the program is {end - start}")

0
13
generate density...
33
generate density...
103
generate density...
131
generate density...
111
generate density...
129
generate density...
90
generate density...
148
generate density...
194
generate density...
204
generate density...
done.
/content/drive/MyDrive/GCC/Part 1/scene_10_0/GT
output /content/drive/MyDrive/GCC/Part 1/scene_10_0/GT/1531114596.h5
done.
/content/drive/MyDrive/GCC/Part 1/scene_10_0/GT
output /content/drive/MyDrive/GCC/Part 1/scene_10_0/GT/1531114596.h5
done.
/content/drive/MyDrive/GCC/Part 1/scene_10_0/GT
output /content/drive/MyDrive/GCC/Part 1/scene_10_0/GT/1531114596.h5
done.
/content/drive/MyDrive/GCC/Part 1/scene_10_0/GT
output /content/drive/MyDrive/GCC/Part 1/scene_10_0/GT/1531114709.h5
done.
/content/drive/MyDrive/GCC/Part 1/scene_10_0/GT
output /content/drive/MyDrive/GCC/Part 1/scene_10_0/GT/1531114709.h5
done.
/content/drive/MyDrive/GCC/Part 1/scene_10_0/GT
output /content/drive/MyDrive/GCC/Part 1/scene_10_0/GT/1531114709.h5
done.
/content/drive/My